In [1]:
import numpy as np
import pandas as pd
import math
from typing import Tuple

pd.set_option('display.precision', 12)  # Increase decimal precision
pd.set_option('display.width', 300)     # Wider display
pd.set_option('display.max_columns', None)  # Show all column

In [2]:
def divided_differences(points, condition):
    """
    points: list of (x_i, y_i) with x_i strictly increasing
    condition: 1 -> forward (first elements of each column)
               0 -> backward (last elements of each column)
    returns: list of selected divided differences (length = len(points))
    """
    x = np.array([p[0] for p in points], dtype=float)
    y = np.array([p[1] for p in points], dtype=float)
    m = len(points)
    if m == 0:
        return []
    if not np.all(np.diff(x) > 0):
        raise ValueError("x values must be strictly increasing.")
    table = np.full((m, m), np.nan, dtype=float)
    table[:, 0] = y.copy()
    for j in range(1, m):
        for i in range(0, m - j):
            table[i, j] = (table[i+1, j-1] - table[i, j-1]) / (x[i+j] - x[i])

    # build DataFrame for display
    data = {'x_i': x, 'y_i': table[:, 0]}
    for j in range(1, m):
        col_vals = [table[i, j] if i < m - j else np.nan for i in range(m)]
        data[f'Order {j}'] = col_vals
    df = pd.DataFrame(data)

    # extract forward or backward selection
    result = []
    for j in range(m):
        col = table[:m - j, j]
        result.append(col[0] if condition == 1 else col[-1])
    return df, result

In [3]:
points = [(0.87, 2.4), (1.24, 2.2), (2.99, 2.0), (3.67, 1.8), (4.23, 1.6)]

df, result = divided_differences(points, condition = 1)

df.style

,x_i,y_i,Order 1,Order 2,Order 3,Order 4
0,0.870000,2.400000,-0.540541,0.201064,-0.098239,0.031545
1,1.240000,2.200000,-0.114286,-0.074005,0.007752,nan
2,2.990000,2.000000,-0.294118,-0.050827,nan,nan
3,3.670000,1.800000,-0.357143,nan,nan,nan
4,4.230000,1.600000,nan,nan,nan,nan


In [4]:
print(result)

[np.float64(2.4), np.float64(-0.5405405405405398), np.float64(0.20106359729001197), np.float64(-0.09823875282008207), np.float64(0.03154483190122133)]


# Newton Interpolation

## Algorithm


- Divided Differences (Coefficients $\mathbf{D}$)

Given $m+1$ points $\{(x_i, y_i)\}_{i=0}^m$:

1.  **Compute $j$-th order difference:**
    $$f[x_i, \dots, x_{i+j}] = \frac{f[x_{i+1}, \dots, x_{i+j}] - f[x_i, \dots, x_{i+j-1}]}{x_{i+j} - x_i} \quad \text{for } j \ge 1$$
2.  **Base Case (0-th order):**
    $$f[x_i] = y_i$$
3.  **Select Interpolation Coefficients ($\mathbf{D}$):**
    * **Forward Newton (Condition 1):**
        $$D_i = f[x_0, x_1, \dots, x_i] \quad \text{for } i=0, \dots, m$$
    * **Backward Newton (Condition 0):**
        $$D_i = f[x_{m-i}, x_{m-i+1}, \dots, x_m] \quad \text{for } i=0, \dots, m$$

- Polynomial Construction

The Newton interpolation polynomial $P_m(x)$ is given by:
$$P_m(x) = \sum_{i=0}^{m} D_i \cdot B_i(x)$$
where $B_i(x)$ is the basis polynomial.

1.  **Basis Polynomial $B_i(x)$:**
    * **Forward Form:** The nodes used are $x_0, x_1, \dots, x_{i-1}$.
        $$B_0(x) = 1$$
        $$B_i(x) = \prod_{k=0}^{i-1} (x - x_k) \quad \text{for } i \ge 1$$
    * **Backward Form:** The nodes used are $x_m, x_{m-1}, \dots, x_{m-(i-1)}$.
        $$B_0(x) = 1$$
        $$B_i(x) = \prod_{k=0}^{i-1} (x - x_{m-k}) \quad \text{for } i \ge 1$$
        * *(Note: The code handles the backward form by reversing $x$ and $D$, effectively using the forward structure on the reversed data.)*

2.  **Expansion (Coefficient Generation):**
    * Iteratively compute the expanded coefficients of $P_m(x)$ by performing the polynomial multiplication $B_i(x) = (x - x_k) B_{i-1}(x)$ and accumulating the components $D_i B_i(x)$.
    * Final result is the array of coefficients $\mathbf{N_{coeff}}$ for the standard form:
        $$P_m(x) = a_0 + a_1 x + a_2 x^2 + \dots + a_m x^m$$

In [5]:
def newton_interpolation(points, condition):
    """
    Build Newton interpolation polynomial coefficients (lowest -> highest).
    Returns numpy array of coefficients [a0, a1, ..., a_n] (constant first).
    """
    m = len(points)
    if m == 0:
        return np.array([])
    x_arr = np.array([p[0] for p in points], dtype=float)
    if not np.all(np.diff(x_arr) > 0):
        raise ValueError("x values must be strictly increasing for the input points.")
    
    # get divided differences (forward or backward)
    tmp, D_list = divided_differences(points, condition=condition)
    # for backward Newton, reverse x and D so loop is same shape
    if condition == 0:
        D_list = D_list[::-1]
        x_arr = x_arr[::-1]
    N_coeff = np.zeros(1, dtype=float)
    steps = []
    for i in range(m):
        D_i = float(D_list[i])
        # build B_{i-1}(x) using lowest-first coefficients
        if i == 0:
            B = np.array([1.0], dtype=float)
        else:
            B = np.array([1.0], dtype=float)
            for k in range(i):
                # (x - x_k) * B  -> x*B - x_k*B
                xB = np.concatenate(([0.0], B))               # x * B (length len(B)+1)
                aB = np.concatenate((x_arr[k] * B, [0.0]))    # x_k * B padded to same length
                B = xB - aB
        N_i = D_i * B
        # add to total polynomial (pad if needed)
        if len(N_coeff) < len(N_i):
            N_coeff = np.pad(N_coeff, (0, len(N_i) - len(N_coeff)), constant_values=0.0)
        N_coeff[:len(N_i)] += N_i
        steps.append({
            'i': i,
            'D_i': D_i,
            'B_(i-1) coeffs (low->high)': np.round(B, 8).tolist(),
            'N_i coeffs (low->high)': np.round(N_i, 8).tolist()
        })

    step_pd = pd.DataFrame(steps)   
    coeff_pd = pd.DataFrame({'Degree': list(range(len(N_coeff))), 'Coeff': N_coeff})

    return step_pd, coeff_pd


## Result

In [6]:
points = [(0.87, 2.4), (1.24, 2.2), (2.99, 2.0), (3.67, 1.8), (4.23, 1.6)]
step_pd, coeff_pd = newton_interpolation(points, condition=1)

In [7]:
step_pd.style

,i,D_i,B_(i-1) coeffs (low->high),N_i coeffs (low->high)
0,0,2.400000,[1.0],[2.4]
1,1,-0.540541,"[-0.87, 1.0]","[0.47027027, -0.54054054]"
2,2,0.201064,"[1.0788, -2.11, 1.0]","[0.21690741, -0.42424419, 0.2010636]"
3,3,-0.098239,"[-3.225612, 7.3877, -5.1, 1.0]","[0.3168801, -0.72575843, 0.50101764, -0.09823875]"
4,4,0.031545,"[11.83799604, -30.338471, 26.1047, -8.77, 1.0]","[0.3734276, -0.95702197, 0.82346837, -0.27664818, 0.03154483]"


In [8]:
coeff_pd.style

,Degree,Coeff
0,0,3.777485
1,1,-2.647565
2,2,1.525550
3,3,-0.374887
4,4,0.031545


In [9]:
#Horner Test
def synthetic_division(a, c):
    """
    Perform synthetic division for polynomial p(x) with coefficients a,
    evaluated at x = c.

    Parameters:
        a (list[float]): coefficients of p(x) from highest to lowest degree
        c (float): the value to evaluate p(c)

    Returns:
        df (pd.DataFrame): table with columns [i, a_i, b_i*c, b_i]
        p_c (float): value of p(c)
        q_coeff (list[float]): coefficients of q(x) = (p(x) - p(c)) / (x - c)
    """

    n = len(a) - 1
    b = [0.0] * (n + 1)
    bc_values = [""] * (n + 1)

    b[n] = a[n]
    for i in range(n - 1, -1, -1):
        b[i] = a[i] + c * b[i + 1]
        bc_values[i + 1] = b[i + 1] * c

    # Prepare table (i from n to 0)
    df = pd.DataFrame({
        "i": list(range(n, -1, -1)),
        "a_i": [a[i] for i in range(n, -1, -1)],
        "b_i*c": [bc_values[i] for i in range(n, -1, -1)],
        "b_i = a_i + b_(i+1)*c": [b[i] for i in range(n, -1, -1)]
    })

    p_c = b[0]
    q_coeff = b[1:]
    return df, p_c, q_coeff, b

In [10]:
def all_derivatives(a, c):
    """
    Compute all derivatives p^(i)(c) using repeated Horner division
    and display in transposed table format.
    """
    coeffs = a.copy()
    degree = len(a) - 1
    results = []
    b0_list = []
    derivative_list = []

    # Perform repeated synthetic division
    for i in range(degree + 1):
        df, b0, next_coeff, b_all = synthetic_division(coeffs, c)
        results.append(b_all)
        b0_list.append(b0)
        derivative_list.append(b0 * math.factorial(i))
        coeffs = next_coeff
        if len(coeffs) == 0:
            break

    # Pad b_i lists for equal column length
    max_len = max(len(b) for b in results)
    for b in results:
        b.extend([None] * (max_len - len(b)))

    # Create DataFrame horizontally
    df = pd.DataFrame(results).T
    df.columns = [f"i={i}" for i in range(len(results))]

    # Insert first column for original a coefficients
    a_col = a + [None] * (df.shape[0] - len(a))
    df.insert(0, "a_i", a_col)

    # Add b_0 and p^(i)(c) rows
    df.loc["b_0"] = [None] + b0_list
    df.loc["p^(i)(c)"] = [None] + derivative_list

    # Add a row on top showing the value of c
    df.loc["c"] = [c] + [None] * (df.shape[1] - 1)
    df = df.loc[["c"] + [idx for idx in df.index if idx != "c"]]  # Move row to top

    return df

In [ ]:
coeff_list = coeff_pd['Coeff'].tolist()
df2 = all_derivatives(coeff_list, 3.5)
df2.style

,a_i,i=0,i=1,i=2,i=3,i=4
c,3.500000,nan,nan,nan,nan,nan
0,3.777485,1.859409,-0.335874,-0.092218,0.066741,0.031545
1,-2.647565,-0.548022,0.060614,-0.043666,0.031545,nan
2,1.525550,0.599870,-0.154073,0.031545,nan,nan
3,-0.374887,-0.264480,0.031545,nan,nan,nan
4,0.031545,0.031545,nan,nan,nan,nan
b_0,nan,1.859409,-0.335874,-0.092218,0.066741,0.031545
p^(i)(c),nan,1.859409,-0.335874,-0.184436,0.400444,0.757076
